### Conclusion

Hypothèse:
- Toute la capacité n'est accessible au début et au fur et à mesure du temp le buffeur diminu sur les MT336 et MT337

## data collect

In [ ]:
from transform.processed_tss.ProcessedTimeSeries import *
import pandas as pd
from core.pandas_utils import *
from transform.raw_results.config import *
import plotly.express as px
from transform.raw_results.tesla_results import get_results

In [ ]:
from transform.processed_results.main import get_processed_results

In [ ]:
df = get_processed_results('tesla')

In [ ]:
from core.sql_utils import *
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join battery b
            on b.id = vm.battery_id
            WHERE vm.model_name like '%model%';"""), con)

dbeaver_df.columns

In [ ]:
df = df.merge(dbeaver_df.groupby('vin', as_index=False)["version"].first().rename(columns={'version':'tesla_code'}), on='vin', how='left')

In [ ]:
df_mt336 = df[df['tesla_code']=="MT336"]

In [ ]:
px.scatter(df_mt336, y='soh', x='odometer', color='vin')

Problèmes les SoH n'évoluent pas au cours du temps 

#On regarde avant que les résultats ne soit filtrés 

In [ ]:
results = get_results()
results_mt336 = results[results["tesla_code"]=="MT336"].copy()
results_mt322 = results[results["tesla_code"]=="MT337"].copy()


In [ ]:
results_mt322[results_mt322['vin'].isin(['LRW3E7FA9MC343785', 'LRW3E7FJ3MC289710'])][['trimmed_in_charge_idx','energy_added_min',
                                                                                      'energy_added_end','soc_diff', 'capacity', 
                                                                                      'charging_power','level_1','level_2',	'level_3']]

In [ ]:
px.scatter(results_mt336, x='odometer', y='soh',  color='vin')

On peut voir que le problème vient surement du calcul

In [ ]:
processed_ts = TeslaProcessedTimeSeries("tesla", force_update=True)

In [ ]:
processed_ts_mT336 = processed_ts[processed_ts['tesla_code']=='MT336']

In [ ]:
processed_ts_mT336_gb = processed_ts_mT336[processed_ts_mT336['trimmed_in_charge']==True].groupby(["vin", "trimmed_in_charge_idx"], observed=True, as_index=False).agg(
    soc_diff=pd.NamedAgg("soc", series_start_end_diff),
    energy_added_min=pd.NamedAgg("charge_energy_added", "min"),
    energy_added_end=pd.NamedAgg("charge_energy_added", "last"),
    capacity=pd.NamedAgg("capacity", 'last'),
    odometer=('odometer', 'last'),
    charging_power=pd.NamedAgg("charging_power", "median"),

) .eval("energy_added = energy_added_end - energy_added_min").eval("soh = energy_added / (soc_diff / 100.0 * capacity)")


In [ ]:
df[(df['vin']=='LRW3E7FA0LC098003')]

In [ ]:
df[(df['vin']=='LRW3E7FAXLC114482')]

In [ ]:
processed_ts_mT336_gb.vin.nunique()

In [ ]:
px.scatter(processed_ts_mT336_gb[(processed_ts_mT336_gb["soh"] < 2) 
                                 & (processed_ts_mT336_gb["soh"] > .5) 
                                 & (processed_ts_mT336_gb["soc_diff"] > 40)].sample(1000), x='odometer', y='soh', trendline='ols', trendline_color_override='red')

In [ ]:
from transform.raw_tss.tesla_raw_tss import get_raw_tss

In [ ]:
tss = get_raw_tss()
tss_mt336 = tss[tss['vin'].isin(df_mt336.vin.unique())]
tss_mt336_charging = tss_mt336[tss_mt336['charging_state']=='Charging'].copy()

In [ ]:
processed_ts_mT336[(processed_ts_mT336['vin']=="LRW3E7FA0LC098003") & (processed_ts_mT336['in_discharge']==False) & (processed_ts_mT336['trimmed_in_charge_idx']==158)].describe()[['soc',
       'battery_range', 'charge_current_request', 'charge_current_request_max',
        'charge_energy_added', 'charge_limit_soc', 'charge_limit_soc_min',
       'charge_miles_added_ideal', 'charge_miles_added_rated', 'charging_rate',
       'charger_actual_current', 'charger_pilot_current', 'charging_power',
       'charger_voltage', 'est_battery_range', 'odometer', 'inside_temp',
       'outside_temp', 'time_diff', 'capacity']]

In [ ]:
processed_ts_mT336[(processed_ts_mT336['vin']=="LRW3E7FA5LC098529") & (processed_ts_mT336['in_discharge']==False) & (processed_ts_mT336['trimmed_in_charge_idx']==167)].describe()[['soc',
       'battery_range', 'charge_current_request', 'charge_current_request_max',
        'charge_energy_added', 'charge_limit_soc', 'charge_limit_soc_min',
       'charge_miles_added_ideal', 'charge_miles_added_rated', 'charging_rate',
       'charger_actual_current', 'charger_pilot_current', 'charging_power',
       'charger_voltage', 'est_battery_range', 'odometer', 'inside_temp',
       'outside_temp', 'time_diff', 'capacity']]

In [ ]:
px.scatter(results_mt336, x='odometer', y='soh', color='vin')